<div style="clear: both;">
<div>
      <h2><b>EXPORT</b>4PHOBS</h2>
     <img src="https://www.adria-ankaran.si//app/uploads/2025/10/logo-Adria.jpg" width="180" alt="">
  </div>
</div>



In [ ]:
from IPython.display import IFrame
IFrame(src='https://docs.google.com/spreadsheets/d/15HJ7wxyUmo-gcl5_y1M9gl4Ti-JSsYEJZCjoI76s-Xk/edit?gid=1385640257#gid=1385640257', width=1200, height=600)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

# ---------- Output container ----------
main_out = widgets.Output()

# ---------- Reload button ----------
reload_button = widgets.Button(
    description="Reload Content 🔄",
    button_style='warning',
    tooltip="Click to refresh all content",
    layout=widgets.Layout(width='200px')
)

# ---------- Function to generate dynamic content ----------
def create_widgets():
    # Format time as HH:MM:SS
    current_time = datetime.now().strftime("%H:%M:%S")
    w1 = widgets.Label(f"Refreshed at: {current_time}")
    return [w1]

# ---------- Function to populate content ----------
def initialize_content():
    with main_out:
        clear_output()
        widgets_list = create_widgets()
        for w in widgets_list:
            display(w)

# ---------- Button click callback ----------
def on_reload_click(b):
    initialize_content()

reload_button.on_click(on_reload_click)

# ---------- Display everything ----------
ui = widgets.VBox([reload_button, main_out])
display(ui)

# Initialize content
initialize_content()


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import base64
import urllib.parse
from datetime import datetime

# ---------- Output container ----------
main_out = widgets.Output()

# ---------- Reload button ----------
reload_button = widgets.Button(
    description="Reload Content 🔄",
    button_style='warning',
    tooltip="Click to refresh all content",
    layout=widgets.Layout(width='200px')
)

# ---------- Last refreshed label ----------
refresh_label = widgets.Label("Last refreshed at: --:--")

# ---------- Spinner output ----------
spinner_out = widgets.Output()

# ---------- Function to prepare PHOBS dataframe ----------
def prepare_phobs_csv(df, hotel_id, los_code):
    if 'BAR' not in df.columns:
        df['BAR'] = 120
    df['BAR'] = df['BAR'].apply(lambda x: f"BAR{x}")
    df['Hotel_ID'] = hotel_id
    df['nicla'] = 0
    df['Yield'] = f"YIELD{los_code}"
    if 'Datum' not in df.columns:
        df['Datum'] = pd.Timestamp.today().strftime('%Y-%m-%d')
    df = df[['Hotel_ID', 'Datum', 'nicla', 'BAR', 'Yield']]
    return df

# ---------- Function to create a download button ----------
def create_download_button(df, filename, description):
    button = widgets.Button(
        description=description,
        button_style='info',
        layout=widgets.Layout(width="33%")
    )
    def on_click(b):
        with main_out:
            main_out.clear_output(wait=True)
            csv_bytes = df.to_csv(index=False, header=False).encode()
            b64 = base64.b64encode(csv_bytes).decode()
            href = f"data:text/csv;base64,{b64}"
            html = f"""
            <div style='margin-top:10px;'>
                <a download="{filename}.csv" href="{href}" target="_blank"
                   style='text-decoration:none; color:white; background-color:#5392ca; padding:6px 10px; border-radius:4px;'>
                   📥 Download {filename}.csv
                </a>
            </div>
            """
            display(HTML(html))
    button.on_click(on_click)
    return button

# ---------- Spinner HTML ----------
spinner_html_content = """
<div style="display:flex; align-items:center; gap:10px; margin:10px 0;">
    <div style="border: 4px solid #f3f3f3; border-top: 4px solid #3498db; border-radius: 50%; width: 18px; height: 18px; animation: spin 1s linear infinite;"></div>
    <span>Loading hotel sheets...</span>
</div>
<style>
@keyframes spin {
  0% { transform: rotate(0deg);}
  100% { transform: rotate(360deg);}
}
</style>
"""

# ---------- Function to initialize PHOBS buttons ----------
def initialize_phobs():
    with main_out:
        main_out.clear_output()
        # Update refresh label
        current_time = datetime.now().strftime("%H:%M")
        refresh_label.value = f"Last refreshed at: {current_time}"
        display(refresh_label)
        display(HTML("<h3>PHOBS BAR Export Generator</h3>"))

        # Display spinner
        with spinner_out:
            clear_output()
            display(HTML(spinner_html_content))
        display(spinner_out)

        # Load master PHOBS sheet
        gsheet_id = "15HJ7wxyUmo-gcl5_y1M9gl4Ti-JSsYEJZCjoI76s-Xk"
        master_sheet_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet=PHOBS"
        master_df = pd.read_csv(master_sheet_url)
        hotels = [(row['Hotel_Name'], row['Hotel_ID'], row['YIELD_Code']) for _, row in master_df.iterrows()]

        buttons = []
        for hotel_name, hotel_id, los_code in hotels:
            try:
                sheet_name_encoded = urllib.parse.quote(hotel_name)
                sheet_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name_encoded}"
                df = pd.read_csv(sheet_url)
                df = prepare_phobs_csv(df, hotel_id, los_code)
                button = create_download_button(df, f"{hotel_name}-PHOBS", hotel_name.replace("_", " "))
                buttons.append(button)
            except Exception as e:
                display(HTML(f"<b style='color:red'>Failed to load sheet for {hotel_name}:</b> {e}"))

        # Remove spinner
        spinner_out.clear_output()

        # Layout buttons in rows of 3
        rows = [widgets.HBox(buttons[i:i+3]) for i in range(0, len(buttons), 3)]
        vbox = widgets.VBox(rows)
        display(vbox)

# ---------- Button click callback ----------
reload_button.on_click(lambda b: initialize_phobs())

# ---------- Display everything ----------
ui = widgets.VBox([reload_button, main_out])
display(ui)

# Initialize content on first load
initialize_phobs()


zadnja verzija

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import base64
import urllib.parse
from datetime import datetime

# --------------------------
# GLOBAL WIDGETS & OUTPUTS
# --------------------------
main_out = widgets.Output()
spinner_out = widgets.Output()
refresh_label = widgets.Label("Last refreshed at: --:--")

reload_button = widgets.Button(
    description="🔄 Reload Content",
    button_style="warning",
    tooltip="Click to refresh all content",
    layout=widgets.Layout(width="200px")
)

# --------------------------
# SPINNER HTML
# --------------------------
spinner_html = """
<div style="display:flex; align-items:center; gap:10px; margin:10px 0;">
  <div style="border: 4px solid #f3f3f3; border-top: 4px solid #3498db;
              border-radius: 50%; width: 18px; height: 18px;
              animation: spin 1s linear infinite;"></div>
  <span>Loading hotel sheets...</span>
</div>
<style>
@keyframes spin {
  0% { transform: rotate(0deg);}
  100% { transform: rotate(360deg);}
}
</style>
"""

# --------------------------
# HELPER FUNCTIONS
# --------------------------
def prepare_phobs_csv(df, hotel_id, los_code):
    """Clean and format PHOBS data."""
    if 'BAR' not in df.columns:
        df['BAR'] = 120
    df['BAR'] = df['BAR'].apply(lambda x: f"BAR{x}")
    df['Hotel_ID'] = hotel_id
    df['nicla'] = 0
    df['Yield'] = f"YIELD{los_code}"
    if 'Datum' not in df.columns:
        df['Datum'] = pd.Timestamp.today().strftime('%Y-%m-%d')
    return df[['Hotel_ID', 'Datum', 'nicla', 'BAR', 'Yield']]

def create_download_button(df, filename, description):
    """Create a clickable button to download the DataFrame as CSV."""
    button = widgets.Button(
        description=description,
        button_style='info',
        layout=widgets.Layout(width="33%")
    )

    def on_click(b):
        temp_out = widgets.Output()
        with temp_out:
            # Convert DataFrame to CSV → base64 → HTML link
            csv_bytes = df.to_csv(index=False, header=False, encoding='utf-8-sig').encode('utf-8-sig')
            b64 = base64.b64encode(csv_bytes).decode()
            href = f"data:text/csv;base64,{b64}"
            html = f"""
            <div id="download-container" style='margin-top:10px;'>
                <a id="download-link" download="{filename}.csv" href="{href}" target="_blank"
                   style='text-decoration:none; color:white; background-color:#5392ca;
                          padding:6px 10px; border-radius:4px;'>
                   📥 Download {filename}.csv
                </a>
            </div>
            <script>
            const link = document.getElementById('download-link');
            link.addEventListener('click', () => {{
                setTimeout(() => {{
                    const container = document.getElementById('download-container');
                    if (container) container.remove();
                }}, 500);
            }});
            </script>
            """
            from IPython.display import HTML
            display(HTML(html))
        display(temp_out)

    button.on_click(on_click)
    return button

# --------------------------
# MAIN INITIALIZATION LOGIC
# --------------------------
def initialize_phobs():
    """Load hotel data and create download buttons."""
    with main_out:
        main_out.clear_output()
        current_time = datetime.now().strftime("%H:%M")
        refresh_label.value = f"Last refreshed at: {current_time}"
        display(refresh_label)

        with spinner_out:
            clear_output()
            from IPython.display import HTML
            display(HTML(spinner_html))
        display(spinner_out)

        try:
            # Load master sheet
            gsheet_id = "15HJ7wxyUmo-gcl5_y1M9gl4Ti-JSsYEJZCjoI76s-Xk"
            master_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet=PHOBS"
            master_df = pd.read_csv(master_url)

            hotels = [(r['Hotel_Name'], r['Hotel_ID'], r['YIELD_Code'])
                      for _, r in master_df.iterrows()]

            buttons = []
            errors = []

            for hotel_name, hotel_id, los_code in hotels:
                try:
                    encoded_name = urllib.parse.quote(hotel_name)
                    url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={encoded_name}"
                    df = pd.read_csv(url)
                    df = prepare_phobs_csv(df, hotel_id, los_code)
                    button = create_download_button(df, f"{hotel_name}-PHOBS", hotel_name.replace("_", " "))
                    buttons.append(button)
                except Exception as e:
                    errors.append((hotel_name, str(e)))

            spinner_out.clear_output()

            # Arrange buttons in rows of 3
            if buttons:
                rows = [widgets.HBox(buttons[i:i+3]) for i in range(0, len(buttons), 3)]
                display(widgets.VBox(rows))

            if errors:
                html_errors = "<h4 style='color:red'>Failed to load some sheets:</h4>"
                for name, msg in errors:
                    html_errors += f"<b>{name}:</b> {msg}<br>"
                from IPython.display import HTML
                display(HTML(html_errors))

        except Exception as e:
            spinner_out.clear_output()
            from IPython.display import HTML
            display(HTML(f"<b style='color:red'>Error loading PHOBS master sheet:</b> {e}"))

# --------------------------
# EVENT BINDING
# --------------------------
reload_button.on_click(lambda b: initialize_phobs())

# --------------------------
# BUILD UI
# --------------------------
header_html = widgets.HTML("<h3 style='margin-top:10px;'>PHOBS BAR Export Generator</h3>")

ui = widgets.VBox([
    header_html,
    reload_button,
    main_out
])

# --------------------------
# DISPLAY ENTRYPOINT
# --------------------------
display(ui)
initialize_phobs()
